# Исследование для сервиса по чтению книг по подписке

**Цель**: Провести исследование базы данных книг для дальнейшей формулировки предложения для нового продукта.  
  
**Задачи**:  
  
- Посчитать, сколько книг вышло после 1 января 2000 года  
- Для каждой книги посчитать количество обзоров и среднюю оценку  
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, таким образим исключить из анализа брошюры  
- Определить автора с самой высокой средней оценкой книг учитывая только книги с 50 и более оценками  
- Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок  


**Этапы исследования**:  
  
1. Загрузить и исследовать данные
2. Выполнить поставленные задачи:  
  
2.1 Посчитать, сколько книг вышло после 1 января 2000 года  
2.2 Для каждой книги посчитать количество обзоров и среднюю оценку  
2.3 Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, таким образим исключить из анализа брошюры  
2.4 Определить автора с самой высокой средней оценкой книг учитывая только книги с 50 и более оценками  
2.5 Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок 
  
3. Сделать выводы

## Загрузим и исследуем данные

In [1]:
# импортируем библиотеки

import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры

db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])

# сохраняем коннектор

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [7]:
query = ''' SELECT * 
                FROM books
LIMIT 5 '''
pd.io.sql.read_sql(query, con = engine) 

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [8]:
query = ''' SELECT * 
                FROM authors    
LIMIT 5 '''

pd.io.sql.read_sql(query, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [9]:
query = ''' SELECT * 
                FROM publishers    
LIMIT 5 '''

pd.io.sql.read_sql(query, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [10]:
query = ''' SELECT * 
                FROM ratings    
LIMIT 5 '''

pd.io.sql.read_sql(query, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [11]:
query = ''' SELECT * 
                FROM reviews    
LIMIT 5 '''

pd.io.sql.read_sql(query, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


В нашем распоряжении находятся 5 баз данных:  
  
**Таблица `books`**

Содержит данные о 1000 книг:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные о 636 авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные о 340 издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о 6456 пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о 2793 пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Выполним задачи

### Посчитаем, сколько книг вышло после 1 января 2000 года  

In [13]:
query = ''' SELECT COUNT(DISTINCT book_id) AS number_of_books
                FROM books
                WHERE publication_date > '2000-01-01' '''   

pd.io.sql.read_sql(query, con = engine)

,number_of_books
0,819


Большая часть книг в базе, 81.9%, вышло после 1 января 2000 года.  

### Для каждой книги посчитаем количество обзоров и среднюю оценку 

In [14]:
query ='''
SELECT
    books.title AS name,
    COUNT(DISTINCT reviews.review_id) AS number_reviews,
    AVG(ratings.rating) AS average_rating
FROM
    books
    INNER JOIN reviews ON reviews.book_id = books.book_id
    INNER JOIN ratings ON ratings.book_id = books.book_id
GROUP BY
    books.title
LIMIT 10 '''

pd.io.sql.read_sql(query, con = engine)

,name,number_reviews,average_rating
0,'Salem's Lot,2,3.666667
1,1 000 Places to See Before You Die,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,4.666667
3,1491: New Revelations of the Americas Before C...,2,4.500000
4,1776,4,4.000000
5,1st to Die (Women's Murder Club #1),4,3.500000
6,2nd Chance (Women's Murder Club #2),3,3.000000
7,4th of July (Women's Murder Club #4),3,4.000000
8,A Beautiful Mind,2,4.250000
9,A Bend in the Road,3,3.200000


У книг с количеством обзоров менее 10 обзоры и средний рейтинг можно считать довольно предвзятыми.  

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц, таким образим исключим из анализа брошюры  

In [15]:
query ='''
SELECT
    publishers.publisher AS publishing_house,
    COUNT(books.num_pages > 50) AS counts
FROM
    books
    INNER JOIN publishers ON publishers.publisher_id = books.publisher_id

GROUP BY
    publishers.publisher
ORDER BY
    counts DESC

LIMIT 5 '''

pd.io.sql.read_sql(query, con = engine)

,publishing_house,counts
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Penguin Books - издательство, которое выпустило наибольшее число книг толще 50 страниц.  

### Определим автора с самой высокой средней оценкой книг учитывая только книги с 50 и более оценками

In [16]:
query = '''
select 
       authors.author,
       authors.author_id,
       avg(ratings.rating) as avg_rating
       
FROM
    books
    INNER JOIN reviews ON reviews.book_id = books.book_id
    INNER JOIN ratings ON ratings.book_id = books.book_id
    INNER JOIN authors ON authors.author_id = books.author_id

where authors.author_id in (select 
       books.author_id
       
from books

where books.book_id in (select 
            ratings.book_id 
            
    from ratings
    group by ratings.book_id
    having count(ratings.rating_id) > 50
    ))
group by authors.author, authors.author_id     
order by avg_rating DESC 
LIMIT 5 '''

pd.io.sql.read_sql(query, con = engine)

,author,author_id,avg_rating
0,J.K. Rowling/Mary GrandPré,236,4.293466
1,Markus Zusak/Cao Xuân Việt Khương,402,4.264151
2,J.R.R. Tolkien,240,4.232892
3,Louisa May Alcott,376,4.196970
4,Rick Riordan,498,4.124481


Автор с самой высокой средней оценкой книг при учете только книг с 50 и более оценками - J.K. Rowling и Mary GrandPré.  
Средний рейтинг 4.29  

### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок 

In [17]:
query = '''
SELECT
     SUM(Sub.count_reviews)/COUNT(Sub.count_reviews) AS count

FROM(    
    select reviews.username, 
    count(reviews.username) as count_reviews
    from reviews
    where reviews.username in (
                                select ratings.username
                                from ratings
                                group by ratings.username
                                having count(ratings.rating_id) > 50
    
                                )
    group by username) as Sub '''

pd.io.sql.read_sql(query, con = engine)

,count
0,24.333333


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.33

## Сделаем выводы

Большая часть книг в базе, 81.9%, вышло после 1 января 2000 года.  
У книг с количеством обзоров менее 10 обзоры и средний рейтинг можно считать довольно предвзятыми.    
Penguin Books - издательство, которое выпустило наибольшее число книг толще 50 страниц.  
Автор с самой высокой средней оценкой книг при учете только книг с 50 и более оценками - J.K. Rowling и Mary GrandPré.  
Среднее количество обзоров от пользователей, которые поставили больше 50 оценок - 24.33
